# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [63]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import warnings
warnings.filterwarnings('ignore')

We set the random seed so that the results don't vary drastically.

In [64]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)
from scipy import ndimage

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [65]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 46 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [66]:
def generator(source_path, folder_list, batch_size,image_size=(120,120),skip=1):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = list(range(0,30,skip))#create a list of image numbers you want to use for a particular video
    x = len(img_idx)
    y,z = image_size
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    image = imresize(image,(y,z,3))
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/np.max(image[:,:,0])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/np.max(image[:,:,1])##normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/np.max(image[:,:,2])##normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        
        # write the code for the remaining data points which are left after full batches
        if num_batches*batch_size < len(folder_list):
            folders_left = t[num_batches*batch_size+1:]
            new_batch_size = len(folders_left)
            batch_data = np.zeros((new_batch_size,x,y,z,3))
            batch_labels = np.zeros((new_batch_size,5))
            for folder in range(new_batch_size):
                imgs = os.listdir(source_path+'/'+ folders_left[folder].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ folders_left[folder].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    image = imresize(image,(y,z,3))                    
                    
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/np.max(image[:,:,0])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/np.max(image[:,:,1])##normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/np.max(image[:,:,2])##normalise and feed in the image
                    
                batch_labels[folder, int(folders_left[folder].strip().split(';')[2])] = 1
            yield batch_data, batch_labels
                
        
        


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [67]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [76]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, Reshape,LSTM
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras
from keras.preprocessing.image import ImageDataGenerator
#write your model here

def make_model(input_shape, num_classes, data_augment=False, dropout_prob=0.0):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = inputs
    
    for i in range(2):
        x = Conv3D(32, 5, padding="same")(x)
        x = Activation("relu")(x)
        x = MaxPooling3D(2)(x)
    
    for i in range(2):
        x = Conv3D(16, 3, padding="same")(x)
        x = Activation("relu")(x)
        x = MaxPooling3D([1,2,2])(x)

    x = Flatten()(x)
    
    x = Dropout(0.8)(x)
    x = Dense(100, activation = "relu")(x)

    activation = "softmax"
    units = num_classes
    x = Dropout(0.1)(x)
    outputs = Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)
  
model = make_model(input_shape=(10,120,120,3),num_classes=5)

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [77]:
optimiser = keras.optimizers.Adam(.001)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 10, 120, 120, 3)   0         
_________________________________________________________________
conv3d_45 (Conv3D)           (None, 10, 120, 120, 32)  2624      
_________________________________________________________________
activation_45 (Activation)   (None, 10, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_45 (MaxPooling (None, 5, 60, 60, 32)     0         
_________________________________________________________________
conv3d_46 (Conv3D)           (None, 5, 60, 60, 32)     27680     
_________________________________________________________________
activation_46 (Activation)   (None, 5, 60, 60, 32)     0         
_________________________________________________________________
max_pooling3d_46 (MaxPooling (None, 2, 30, 30, 32)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [78]:
train_generator = generator(train_path, train_doc, batch_size,skip=3)
val_generator = generator(val_path, val_doc, batch_size, skip=3)

In [79]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [80]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [81]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 46
Epoch 1/50
15/15 [==============================] - 35s 2s/step - loss: 1.6157 - categorical_accuracy: 0.2004 - val_loss: 1.6063 - val_categorical_accuracy: 0.2525

Epoch 00001: saving model to model_init_2021-02-0115_23_19.107624/model-00001-1.61477-0.20393-1.60634-0.25253.h5
Epoch 2/50
15/15 [==============================] - 25s 2s/step - loss: 1.6031 - categorical_accuracy: 0.2069 - val_loss: 1.5903 - val_categorical_accuracy: 0.2323

Epoch 00002: saving model to model_init_2021-02-0115_23_19.107624/model-00002-1.60339-0.20846-1.59028-0.23232.h5
Epoch 3/50
15/15 [==============================] - 25s 2s/step - loss: 1.5762 - categorical_accuracy: 0.2192 - val_loss: 1.5203 - val_categorical_accuracy: 0.2222

Epoch 00003: saving model to model_init_2021-02-0115_23_19.107624/model-00003-1.58021-0.21903-1.52031-0.22222.h5
Epoch 4/50
15/15 [==============================] - 28s 2s/step - loss: 1.5429 - categorical_accuracy: 0.2527 - va

15/15 [==============================] - 24s 2s/step - loss: 0.5117 - categorical_accuracy: 0.7830 - val_loss: 0.7028 - val_categorical_accuracy: 0.7071

Epoch 00030: saving model to model_init_2021-02-0115_23_19.107624/model-00030-0.51617-0.78097-0.70275-0.70707.h5
Epoch 31/50
15/15 [==============================] - 28s 2s/step - loss: 0.5102 - categorical_accuracy: 0.7988 - val_loss: 0.7144 - val_categorical_accuracy: 0.7374

Epoch 00031: saving model to model_init_2021-02-0115_23_19.107624/model-00031-0.51882-0.79305-0.71445-0.73737.h5
Epoch 32/50
15/15 [==============================] - 26s 2s/step - loss: 0.5156 - categorical_accuracy: 0.7975 - val_loss: 0.7603 - val_categorical_accuracy: 0.6970

Epoch 00032: saving model to model_init_2021-02-0115_23_19.107624/model-00032-0.51810-0.79607-0.76032-0.69697.h5
Epoch 33/50
15/15 [==============================] - 24s 2s/step - loss: 0.4690 - categorical_accuracy: 0.8156 - val_loss: 0.6858 - val_categorical_accuracy: 0.7475

Epoch 000